In [1]:
import os
import tensorflow as tf
import pandas as pd
tf.config.set_soft_device_placement(False)
tf.debugging.set_log_device_placement(True)
import numpy as np
from ampligraph.datasets import load_fb15k_237
from ampligraph.evaluation.protocol import create_mappings, to_idx

assert(tf.__version__.startswith('2.3'))

from ampligraph.latent_features import EmbeddingModel

In [2]:
fb_dataset = load_fb15k_237()

rel_to_idx, ent_to_idx = create_mappings(fb_dataset['train'])
train_dataset = to_idx(fb_dataset['train'], ent_to_idx, rel_to_idx)
test_dataset = to_idx(fb_dataset['test'], ent_to_idx, rel_to_idx)

In [3]:
batch_size=30000
n_epochs=200
eta=5
    
batch_count = int(np.round(train_dataset.shape[0]/batch_size))


with tf.device('GPU:0'):
    model = EmbeddingModel(eta, ent_to_idx, rel_to_idx, seed=117)

    def nll(scores_dummy, scores_pred):
        return tf.reduce_sum(tf.math.log(1 + tf.exp(-scores_pred[0]))) + \
            tf.reduce_sum(tf.math.log(1 + tf.exp(scores_pred[1])))

    optim = tf.optimizers.Adam(lr=0.001)

    

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalN

In [4]:



def data_gen():
    for j in range(batch_count):
        inputs = train_dataset[j*batch_size:(j+1)*batch_size, :].astype(np.int32)
        yield inputs


@tf.function()
def train_step(inputs, model):
    with tf.GradientTape() as tape:
        preds = model(inputs, training=0)
        scores_dummy=None
        loss = nll(preds, preds)
    gradients = tape.gradient(loss, model.trainable_variables)
    optim.apply_gradients(zip(gradients, model.trainable_variables))   
    return loss
        
        

def train(model):
    dataset = tf.data.Dataset.from_generator(data_gen,
                                         output_types=(tf.int32),
                                         output_shapes=((None, 3)))
    dataset = dataset.prefetch(0)
    print('Batches:', batch_count)
    print('Epochs:', n_epochs)
    for i in range(n_epochs):
        total_loss = []
        for j, inputs in dataset.enumerate():
            with tf.device('GPU:0'):
                loss = train_step(inputs, model)
                #print(loss)
            total_loss.append(loss/inputs.shape[0])
        print('Loss (Epoch{}):{}'.format(i, np.mean(total_loss)))
    print("Model fitted")
    
    return
    



import time
start = time.time()
train(model)
print('done')
print(time.time() - start)

Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Batches: 9
Epochs: 200
Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MakeIterator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
training 0
tr
done
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica

Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_896 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DeleteIterator in device /job:localhost/replica:0/task:0/device:CPU:0
Loss (Epoch0):4.158878803253174
Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MakeIterator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op IteratorGetNext in device /job:localhos

Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_896 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_896 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_896 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_896 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/

Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_896 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_896 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_896 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_896 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/

Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MakeIterator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_896 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_896 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_896 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:

...

Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_896 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DeleteIterator in device /job:localhost/replica:0/task:0/device:CPU:0
Loss (Epoch187):0.23440425097942352
Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MakeIterator in device /job:localhost/re

Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_896 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_896 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_896 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_896 in device /job:localhost/replica:0/

Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_896 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_896 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_896 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_896 in device /job:localhost/replica:0/

Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MakeIterator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_896 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_896 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_896 in device /job:localhost/replica

Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_896 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DeleteIterator in device /job:localhost/replica:0/task:0/device:CPU:0
Loss (Epoch198):0.225648432970047
Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MakeIterator in device /job:localhost/repl

In [5]:
0.22478048503398895

0.22478048503398895

In [6]:
model.save_weights('./checkpoints/saved_model')

Executing op StaticRegexFullMatch in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Select in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op StringJoin in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ShardedFilename in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op SaveV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ShardedFilename in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op SaveV2 in device /job:local

In [7]:
del model

Executing op DestroyResourceOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op DestroyResourceOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op DestroyResourceOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op DestroyResourceOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op DestroyResourceOp in device /job:localhost/replica:0/task:0/device:GPU:0


In [8]:
physical_devices = tf.config.list_physical_devices('GPU')

In [9]:
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [10]:
models = []
for i in range(len(physical_devices)):
    with tf.device('GPU:{}'.format(i)):
        model = EmbeddingModel(eta, ent_to_idx, rel_to_idx)
        model.load_weights('./checkpoints/saved_model')
        models.append(model)

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalN

In [11]:
#test_dataset = test_dataset[:-1, :]
print(test_dataset.shape)

(20438, 3)


In [12]:
import time

batch_size = len(models)
batch_count = np.int32(np.round(test_dataset.shape[0]/batch_size))
#batch_count = 4000//batch_size
def test_data_gen():
    for j in range(batch_count):
        inputs = test_dataset[j * batch_size: (j+1) * batch_size, :].astype(np.int32)
        yield inputs
        

print(batch_count)


@tf.function(autograph=True)
def compute_ranks(preds):
    inp_score = preds[0]
    corr_score = preds[1]
    obj_corruption_scores = tf.slice(corr_score,
                                     [0],
                                     [tf.shape(corr_score)[0] // 2])

    subj_corruption_scores = tf.slice(corr_score,
                                      [tf.shape(corr_score)[0] // 2],
                                      [tf.shape(corr_score)[0] // 2])


    rank = tf.stack([tf.reduce_sum(tf.cast(subj_corruption_scores >= inp_score, 
                                           tf.int32)) + 1,
                     tf.reduce_sum(tf.cast(obj_corruption_scores >= inp_score,
                          tf.int32)) + 1 ], 0)
    return rank


@tf.function(autograph=True)
def test_step(inputs, models):
    ranks = []
    if inputs.shape[0]==1:
        with tf.device('GPU:0'):
            preds = models[0](inputs, training=1)
            rank = compute_ranks(preds)
            ranks.append(rank)
        return ranks
    else:
            
        input1, input2 = tf.split(inputs, 2)

        for i in range(len(models)):
            with tf.device('GPU:{}'.format(i)):
                preds = models[i](input1, training=1)
                rank = compute_ranks(preds)
                ranks.append(rank)

        return ranks


@tf.function(autograph=True)
def test_step1model(inputs, model, device):
    with tf.device('GPU:{}'.format(device)):
        preds1 = model(inputs, training=1)
        ranks1 = compute_ranks(preds1)
    return ranks1


import tqdm



def test(models):
    dataset = tf.data.Dataset.from_generator(test_data_gen,
                                             output_types=(tf.int32),
                                             output_shapes=((None, 3)))

    dataset = dataset.prefetch(0)

    all_ranks = []
    for i, inputs in dataset.enumerate():

        
        

        ranks = test_step(inputs, models)
        
        all_ranks.extend([ranks[i].numpy() for i in range(len(ranks))])

    print('done')
    return np.array(all_ranks)


start=time.time()
ranks = test(models)
print(i, (time.time()-start)/60)




10219
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MakeIterator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
training 1
ev
creating eval--------------------------------
done
Executing op RestoreV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in

Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Exec

Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __i

Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __i

Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __i

...

Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __i

Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __i

Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __i

Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __i

Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_step_12547 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __i

In [13]:
from ampligraph.evaluation import mrr_score, hits_at_n_score
print('Unfiltered eval metrics:')
print('MRR:', mrr_score(ranks))
print('Hits@1:', hits_at_n_score(ranks, 1))

Unfiltered eval metrics:
MRR: 0.08638761082173106
Hits@1: 0.01609746550543106


In [14]:
ranks.shape

(20438, 2)